In [1]:
import pandas as pd

In [2]:
df = pd.read_csv(r"C:\Users\aceradmin\Desktop\ProjectsForPortfolio\netflix_titles.csv")
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [3]:
df.info()
df.describe(include='all')
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [4]:
df.drop(columns=['description'], inplace=True)

In [5]:
# Renaming columns to avoid conflicts with SQL keywords
df.rename(columns={
    'type': 'show_type',
    'cast': 'cast_list'
}, inplace=True)

In [6]:
# Renaming columns to improve clarity
df.rename(columns={
    'country': 'production_country',
    'rating': 'age_rating',
    'listed_in': 'genres'
}, inplace=True)

In [7]:
df.columns

Index(['show_id', 'show_type', 'title', 'director', 'cast_list',
       'production_country', 'date_added', 'release_year', 'age_rating',
       'duration', 'genres'],
      dtype='object')

In [9]:
df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')

In [10]:
# Handling missing values
df = df.dropna(subset=['title'])
df = df.dropna(subset=['duration'])
df = df.dropna(subset=['age_rating'])
df = df.dropna(subset=['date_added'])

In [11]:
df['director'] = df['director'].fillna('Unknown')
df['cast_list'] = df['cast_list'].fillna('Unknown')
df['production_country'] = df['production_country'].fillna('Unknown')

In [12]:
# Double-checking missing values
df.isnull().sum()

show_id               0
show_type             0
title                 0
director              0
cast_list             0
production_country    0
date_added            0
release_year          0
age_rating            0
duration              0
genres                0
dtype: int64

In [13]:
# Extract year and month components
df['year_added'] = df['date_added'].dt.year
df['month_added'] = df['date_added'].dt.month

In [14]:
# Clean and standardize multi-valued string columns by trimming spaces and normalizing comma separators
multi_valued_columns = ['director', 'cast_list', 'production_country', 'genres']

In [15]:
for col in multi_valued_columns:
    df[col] = df[col].astype(str) 
    df[col] = df[col].str.strip()  
    df[col] = df[col].str.replace(r'\s*,\s*', ', ', regex=True)  

In [17]:
# Final Check
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 8702 entries, 0 to 8806
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   show_id             8702 non-null   object        
 1   show_type           8702 non-null   object        
 2   title               8702 non-null   object        
 3   director            8702 non-null   object        
 4   cast_list           8702 non-null   object        
 5   production_country  8702 non-null   object        
 6   date_added          8702 non-null   datetime64[ns]
 7   release_year        8702 non-null   int64         
 8   age_rating          8702 non-null   object        
 9   duration            8702 non-null   object        
 10  genres              8702 non-null   object        
 11  year_added          8702 non-null   int32         
 12  month_added         8702 non-null   int32         
dtypes: datetime64[ns](1), int32(2), int64(1), object(9)
m

show_id               0
show_type             0
title                 0
director              0
cast_list             0
production_country    0
date_added            0
release_year          0
age_rating            0
duration              0
genres                0
year_added            0
month_added           0
dtype: int64

In [18]:
df.to_csv("netflix_titles_cleaned.csv", index=False)